In [1]:
import torch

In [2]:
torch.__version__
torch.version.cuda
torch.backends.cudnn.version()

8500

In [18]:
import json
import pickle
import numpy as np

In [24]:
train_img = "../data/processed/trimmed_mapping_SpeakerALL/train_image_paths.pickle"
train_cap_len = "../data/processed/trimmed_mapping_SpeakerALL/TRAIN_CAPLENS_coco_2_cap_per_img_1_min_word_freq.json"
valid_img = "../data/processed/trimmed_mapping_SpeakerALL/val_image_paths.pickle"
valid_cap_len = "../data/processed/trimmed_mapping_SpeakerALL/VAL_CAPLENS_coco_2_cap_per_img_1_min_word_freq.json"

In [27]:
def read_json(path):
    with open(path, "r") as f:
        data = json.load(f)
    return data
def read_pickle(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

In [26]:
train_img = read_pickle(train_img)
train_cap_len = read_json(train_cap_len)
valid_img = read_pickle(valid_img)
valid_cap_len = read_json(valid_cap_len)
train_avg_len = np.mean(train_cap_len)
valid_avg_len = np.mean(valid_cap_len)

In [28]:
train_img = "../data/processed/encodec1/train_image_paths.pickle"
train_cap_len = "../data/processed/encodec1/TRAIN_CAPLENS_coco_1_cap_per_img_1_min_word_freq.json"
valid_img = "../data/processed/encodec1/val_image_paths.pickle"
valid_cap_len = "../data/processed/encodec1/VAL_CAPLENS_coco_1_cap_per_img_1_min_word_freq.json"

In [29]:
train_img = read_pickle(train_img)
train_cap_len = read_json(train_cap_len)
valid_img = read_pickle(valid_img)
valid_cap_len = read_json(valid_cap_len)
train_avg_len = np.mean(train_cap_len)
valid_avg_len = np.mean(valid_cap_len)

In [35]:
valid_avg_len

275.2640949554896

In [12]:
def generate_VALLE_mask(img_len, seq_len) -> torch.Tensor:
        """
        来自原来的function
        math:`(L, S)` or :math:`(N\cdot\text{num\_heads}, L, S)`, where :math:`N` is the batch size,
        math:`L` is the target sequence length, and :math:`S` is the source sequence length.

        args : 
            img_len : input image_features len. 
                        e.g.: [batch, feature_size, feature_size, dim] -> [batch, feature_size x feature_size, dim]
                        img_len = feature_size x feature_size
            seq_len : input seq len. e.g.: number of tokens.
        e.g.:
        if img_len = 4, seq_len = 5, it'll be like:
            [[0.,   0.,   0.,   0., -inf, -inf, -inf, -inf, -inf],
             [0.,   0.,   0.,   0., -inf, -inf, -inf, -inf, -inf],
             [0.,   0.,   0.,   0., -inf, -inf, -inf, -inf, -inf],
             [0.,   0.,   0.,   0., -inf, -inf, -inf, -inf, -inf],
             [0.,   0.,   0.,   0.,   0., -inf, -inf, -inf, -inf],
             [0.,   0.,   0.,   0.,   0.,   0., -inf, -inf, -inf],
             [0.,   0.,   0.,   0.,   0.,   0.,   0., -inf, -inf],
             [0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., -inf],
             [0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]]
        """
        
        up = torch.full((img_len, img_len + seq_len), float(0))
        up[:,img_len:] = float("-inf")
        down = torch.triu(torch.full((seq_len, img_len + seq_len), float('-inf')), diagonal=img_len+1)

        mask = torch.cat([up, down], dim = 0)
        # 重新设计mask
        return mask

In [13]:
img_len = 1
seq_len = 5
generate_VALLE_mask(img_len, seq_len)

tensor([[0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0.]])